In [226]:
import torch
import torchvision
from torch import nn
from torchsummary import summary
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Setting

In [7]:
plt.style.use("dark_background")

# Model

In [303]:
class BiLSTM(nn.Module):
    def __init__(self, feature_size, hidden_dim, lstm_layer, dropout = 0.2):
        super(BiLSTM, self).__init__()
        self.input_size = feature_size
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(p = dropout)
        self.lstm = nn.LSTM(input_size = self.input_size, 
                            hidden_size = hidden_dim,
                            num_layers = lstm_layer,
                            dropout = dropout,
                            bidirectional = True)
        self.hidden2label = nn.Linear(hidden_dim*2, 8)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return lstm_out
        # label_space = self.hidden2label(lstm_out)
        # label_scores = F.log_softmax(label_space, dim = 1)
        # return label_scores

In [304]:
sample = torch.rand([1, 1000, 32])
net = BiLSTM(32, 128, 1)

In [305]:
pred = net(sample)

In [137]:
pred.squeeze().detach().numpy().argmax(axis = 1) + 1

array([5, 2, 3, 7, 7, 8, 2, 6, 1, 8, 8, 2, 6, 4, 7, 6, 6, 7, 8, 3, 5, 3,
       1, 7, 3, 2, 8, 6, 7, 1, 7, 7, 3, 8, 7, 2, 8, 5, 4, 4, 4, 1, 1, 1,
       5, 3, 6, 4, 8, 6, 7, 1, 1, 2, 5, 4, 6, 3, 2, 3, 7, 6, 7, 5, 8, 7,
       5, 4, 6, 3, 2, 6, 5, 1, 7, 7, 8, 2, 4, 3, 5, 5, 2, 4, 3, 3, 1, 8,
       6, 3, 3, 8, 2, 8, 1, 5, 8, 4, 2, 2, 5, 5, 3, 6, 5, 2, 7, 1, 3, 2,
       3, 4, 6, 4, 4, 2, 3, 2, 4, 1, 7, 1, 2, 7, 8, 3, 2, 1, 7, 8, 7, 5,
       4, 4, 8, 1, 5, 8, 7, 1, 1, 5, 7, 3, 8, 3, 1, 2, 5, 8, 1, 3, 7, 7,
       2, 4, 8, 8, 6, 6, 3, 7, 3, 2, 3, 3, 8, 1, 6, 1, 3, 4, 3, 4, 4, 3,
       3, 7, 2, 2, 1, 8, 7, 6, 3, 3, 1, 8, 2, 4, 4, 6, 2, 2, 2, 3, 2, 8,
       8, 5, 1, 4, 5, 6, 1, 7, 4, 2, 4, 4, 8, 3, 6, 2, 3, 8, 2, 8, 8, 4,
       2, 8, 8, 6, 1, 1, 1, 7, 7, 3, 7, 3, 8, 7, 4, 2, 5, 6, 2, 4, 5, 8,
       4, 3, 8, 5, 4, 2, 8, 7, 2, 3, 7, 8, 7, 4, 8, 8, 3, 3, 7, 6, 7, 5,
       6, 4, 2, 1, 7, 5, 2, 3, 4, 1, 7, 1, 6, 4, 4, 7, 8, 3, 1, 6, 8, 1,
       2, 4, 2, 2, 5, 8, 6, 7, 5, 6, 2, 2, 8, 5, 3,

# Data

In [2]:
data_df = pd.read_csv('data/mid_res/20210526_data_df.csv')
val_df = pd.read_csv('data/mid_res/20210526_val_df.csv')

In [5]:
col_drop_x = ['time', 'label']
col_label = 'label'
col_drop = ['latitude',
 'longitude',
 'altitude',
 'time_dlt',
 'valid_dlt',
 'east',
 'north',
 'east_dlt',
 'north_dlt',
 'east_speed',
 'north_speed',
 'cells_ctype_mode',
 'speed_dif',
 'speed_dlt',
 'speed'
 ]
X_train, y_train = data_df.drop(col_drop_x + col_drop, axis = 1).fillna(0), data_df[col_label]
X_val, y_val = val_df.drop(col_drop_x + col_drop, axis = 1).fillna(0), val_df[col_label]

In [19]:
X_train.shape

(980527, 32)

In [252]:
class TMR_Dataset(Dataset):
    def __init__(self, x_array):
        super().__init__()
        self.len, self.feature_size = x_array.shape
        self.data = x_array
        self.data = torch.tensor(self.data)
    
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.data[idx,:]

In [264]:
BATCH_SIZE = 4

data = TMR_Dataset(np.array(X_train))
data_loader = DataLoader(data, batch_size = BATCH_SIZE, shuffle = False)

In [265]:
x = next(iter(data_loader)).unsqueeze(0)

In [270]:
x.shape, sample.shape

(torch.Size([1, 4, 32]), torch.Size([1, 1000, 32]))

In [271]:
net(sample)

torch.Size([1, 1000, 256])
torch.Size([1, 1000, 8])


tensor([[[-6.9061, -6.9084, -6.9085,  ..., -6.9083, -6.9065, -6.9093],
         [-6.9072, -6.9078, -6.9077,  ..., -6.9078, -6.9066, -6.9091],
         [-6.9077, -6.9086, -6.9086,  ..., -6.9082, -6.9098, -6.9090],
         ...,
         [-6.9062, -6.9069, -6.9086,  ..., -6.9076, -6.9075, -6.9044],
         [-6.9075, -6.9070, -6.9068,  ..., -6.9069, -6.9089, -6.9073],
         [-6.9087, -6.9073, -6.9093,  ..., -6.9076, -6.9072, -6.9083]]],
       grad_fn=<LogSoftmaxBackward>)

tensor([[[  0.0000,   4.0000,   6.0000,   0.0000,   0.0000, -48.0000, -60.6667,
          -90.0000, -43.0000,  17.9518,   0.1667,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000,  22.5000,  16.0000,  29.0000,   5.6862,   0.0000,
            0.0000,   0.0000,   3.2450,   0.0000],
         [  0.0000,   4.0000,   6.0000,   0.0000,   0.0000, -48.0000, -60.6667,
          -90.0000, -43.0000,  17.9518,   0.1667,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000,  22.2500,  15.0000,  29.0000,   6.0759,   0.0000,
            0.0000,   0.0000,   3.2450,   0.0000],
         [  0.0000,   4.0000,   6.0000,   0.0000,   0.0000, -48.0000, -60.6667,
          -90.0000, -43.0000,  17.9518,   0.1667,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
            0.0000